In [66]:
import pandas as pd

# Load the mapping file into a DataFrame
mapping_df = pd.read_csv('test.csv')
df = pd.read_csv('test.csv')

# Check the columns in the DataFrame
print("Columns in the DataFrame:")
print(df.columns)

# Define a function to fill null values in the 'Region' column based on 'patient_state' and 'patient_zip3'
def fill_region(row):
    if pd.isnull(row['Region']):
        if not pd.isnull(row['patient_state']):
            matching_rows = mapping_df.loc[mapping_df['patient_state'] == row['patient_state'], 'Region']
            if not matching_rows.empty:
                return matching_rows.iloc[0]
        if not pd.isnull(row['patient_zip3']):
            # Find the state associated with the patient_zip3
            matching_state_rows = mapping_df.loc[mapping_df['patient_zip3'] == row['patient_zip3'], 'patient_state']
            if not matching_state_rows.empty:
                state = matching_state_rows.iloc[0]
                # Find the region associated with the state
                matching_region_rows = mapping_df.loc[mapping_df['patient_state'] == state, 'Region']
                if not matching_region_rows.empty:
                    return matching_region_rows.iloc[0]
    return row['Region']

# Define a function to fill null values in the 'patient_state' column based on 'patient_zip3' and 'Region'
def fill_state(row):
    if pd.isnull(row['patient_state']):
        if not pd.isnull(row['patient_zip3']):
            matching_rows = mapping_df.loc[mapping_df['patient_zip3'] == row['patient_zip3'], 'patient_state']
            if not matching_rows.empty:
                return matching_rows.iloc[0]
        if not pd.isnull(row['Region']):
            matching_rows = mapping_df.loc[mapping_df['Region'] == row['Region'], 'patient_state']
            if not matching_rows.empty:
                return matching_rows.iloc[0]
    return row['patient_state']

# Define a function to fill null values in the 'patient_zip3' column based on 'patient_state' and 'Region'
def fill_zip(row):
    if pd.isnull(row['patient_zip3']):
        if not pd.isnull(row['patient_state']):
            matching_rows = mapping_df.loc[mapping_df['patient_state'] == row['patient_state'], 'patient_zip3']
            if not matching_rows.empty:
                return matching_rows.iloc[0]
        if not pd.isnull(row['Region']):
            matching_rows = mapping_df.loc[mapping_df['Region'] == row['Region'], 'patient_zip3']
            if not matching_rows.empty:
                return matching_rows.iloc[0]
    return row['patient_zip3']

# Apply the defined functions to fill null values in the DataFrame
df['Region'] = df.apply(fill_region, axis=1)
df['patient_state'] = df.apply(fill_state, axis=1)
df['patient_zip3'] = df.apply(fill_zip, axis=1)

# Handle the case where region, state, and zip are not available but zip is available
for index, row in df.iterrows():
    if pd.isnull(row['Region']) and pd.isnull(row['patient_state']) and pd.isnull(row['patient_zip3']) and not pd.isnull(row['Zip']):
        # Find the state associated with the ZIP code
        matching_state_rows = mapping_df.loc[mapping_df['patient_zip3'] == row['Zip'], 'patient_state']
        if not matching_state_rows.empty:
            state = matching_state_rows.iloc[0]
            # Find the region associated with the state
            matching_region_rows = mapping_df.loc[mapping_df['patient_state'] == state, 'Region']
            if not matching_region_rows.empty:
                df.at[index, 'Region'] = matching_region_rows.iloc[0]

        df.at[index, 'patient_state'] = state
        df.at[index, 'patient_zip3'] = row['Zip']

# Save the updated DataFrame to a new CSV file
df.to_csv('filled_RSZ.csv', index=False)

# Print only the rows where there are missing values in 'patient_state', 'patient_zip3', and 'Region' columns
missing_data = df[df[['patient_state', 'patient_zip3', 'Region']].isnull().any(axis=1)]
print("Data with missing values:")
print(missing_data[['patient_state', 'patient_zip3', 'Region']])

# Check for missing values in the 'patient_zip3', 'patient_state', and 'Region' columns
missing_zip3 = df['patient_zip3'].isnull().sum()
missing_state = df['patient_state'].isnull().sum()
missing_region = df['Region'].isnull().sum()

print("Missing values in 'patient_zip3' column:", missing_zip3)
print("Missing values in 'patient_state' column:", missing_state)
print("Missing values in 'Region' column:", missing_region)


Columns in the DataFrame:
Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'Region', 'Division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_

In [67]:
from google.colab import files
files.download('filled_RSZ.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
import pandas as pd

df = pd.read_csv('filled_RSZ.csv')

# Function to calculate mean BMI based on region, state, or zip code
def calculate_mean_bmi(region, state, zip_code):
    if region != "Unknown" and region in region_bmi_mean:
        return region_bmi_mean[region]
    elif state != "Unknown" and state in state_bmi_mean:
        return state_bmi_mean[state]
    elif zip_code != "Unknown" and zip_code in zip_bmi_mean:
        return zip_bmi_mean[zip_code]
    else:
        return None

# Calculate mean BMI for each combination of region, state, and zip code
region_bmi_mean = df.groupby('Region')['bmi'].mean().to_dict()
state_bmi_mean = df.groupby('patient_state')['bmi'].mean().to_dict()
zip_bmi_mean = df.groupby('patient_zip3')['bmi'].mean().to_dict()

# Fill null values in 'BMI' column based on mean BMI of region, state, or zip code
for index, row in df.iterrows():
    if pd.isnull(row['bmi']):
        mean_bmi = calculate_mean_bmi(row['Region'], row['patient_state'], row['patient_zip3'])
        df.at[index, 'bmi'] = mean_bmi

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_BMI_dataset.csv', index=False)

from google.colab import files
# files.download('submission_FS3.csv')
files.download('updated_BMI_dataset.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
newdf = pd.read_csv('updated_BMI_dataset.csv')

# Specify the column for which you want to check null values
column_name = 'bmi'  # Replace 'BMI' with the column name you want to check

# Check for null values in the specified column
null_values = newdf[column_name].isnull().sum()

# Print the number of null values
print(f"Number of null values in column '{column_name}': {null_values}")

# Get the patient IDs where BMI entries are null
null_bmi_patients = newdf.loc[df[column_name].isnull(), 'patient_id']

# Print the patient IDs
print("Patient IDs with null BMI entries:")
for patient_id in null_bmi_patients:
    print(patient_id)

Number of null values in column 'bmi': 0
Patient IDs with null BMI entries:


In [20]:
import pandas as pd

df = pd.read_csv('filled_RSZ.csv')

# Function to calculate mean BMI based on region, state, or zip code
def calculate_mean_bmi(region, state, zip_code):
    if state != "Unknown" and state in state_bmi_mean:
        return state_bmi_mean[state]
    elif zip_code != "Unknown" and zip_code in zip_bmi_mean:
        return zip_bmi_mean[zip_code]
    else:
        return overall_mean_bmi

# Calculate mean BMI for each combination of state and zip code
state_bmi_mean = df.groupby('patient_state')['bmi'].mean().to_dict()
zip_bmi_mean = df.groupby('patient_zip3')['bmi'].mean().to_dict()

# Calculate overall mean BMI of the entire dataset
overall_mean_bmi = df['bmi'].mean()

# Fill null values in 'BMI' column based on mean BMI of state or zip code
unfilled_patients = []
for index, row in df.iterrows():
    if pd.isnull(row['bmi']):
        mean_bmi = calculate_mean_bmi(row['Region'], row['patient_state'], row['patient_zip3'])
        if mean_bmi is not None:
            df.at[index, 'bmi'] = mean_bmi
        else:
            unfilled_patients.append(row['patient_ID'])

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_BMI_dataset.csv', index=False)

# Print the patient IDs for which BMI values are not filled
print("Patient IDs for which BMI values are not filled:")
print(unfilled_patients)


Patient IDs for which BMI values are not filled:
[]


In [55]:
newdf = pd.read_csv('updated_BMI_dataset.csv')

# Specify the column for which you want to check null values
column_name = 'bmi'  # Replace 'BMI' with the column name you want to check

# Check for null values in the specified column
null_values = newdf[column_name].isnull().sum()

# Print the number of null values
print(f"Number of null values in column '{column_name}': {null_values}")

# Get the patient IDs where BMI entries are null
null_bmi_patients = newdf.loc[df[column_name].isnull(), 'patient_id']

# Print the patient IDs
print("Patient IDs with null BMI entries:")
for patient_id in null_bmi_patients:
    print(patient_id)

Number of null values in column 'bmi': 0
Patient IDs with null BMI entries:
